# IBM Data Science Capstone Week 3 Assignment

In [22]:
import pandas as pd
import os

In [25]:
# I tried scraping the table with wget, beautiful soup, etc., but the table in html was an untitled div, and proved difficult to scrape

#Instead I chose to convert the table to csv

TorontoData = pd.read_csv(r"C:\Users\Todo\Downloads\Canada_Table.csv")

TorontoData.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
#Next attempt at a proper scrape
from bs4 import BeautifulSoup
import requests

wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


In [38]:
class_id = 'wikitable sortable jquery-tablesorter'

response = requests.get(wiki_url)
soup = BeautifulSoup(response.text, 'html')
print(soup.prettify())

nada:_S" title="List of postal codes of Canada: S">
               S
              </a>
             </td>
             <td align="center" style="border: 1px solid #0040FF; background-color: #E0E8FF; font-size: 135%;" width="5%">
              <a href="/wiki/List_of_postal_codes_of_Canada:_T" title="List of postal codes of Canada: T">
               T
              </a>
             </td>
             <td align="center" style="border: 1px solid #0000FF; background-color: #E0E0FF; font-size: 135%;" width="5%">
              <a href="/wiki/List_of_postal_codes_of_Canada:_V" title="List of postal codes of Canada: V">
               V
              </a>
             </td>
             <td align="center" style="border: 1px solid #A000FF; background-color: #E8E0FF; font-size: 135%;" width="5%">
              <a href="/wiki/List_of_postal_codes_of_Canada:_X" title="List of postal codes of Canada: X">
               X
              </a>
             </td>
             <td align="center" style=

In [39]:
canada_table = soup.find("table",{"class": "wikitable sortable"})
print(canada_table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<

In [41]:
df = pd.read_html(str(canada_table))

In [42]:
df

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                           Neighborhood  
 0                                                  NaN  
 1                                                  NaN  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                                NaN  
 176                                                NaN  
 177                

In [ ]:
#I either need to go back and ignore rows with NaN, or clean it from here

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

#After cleaning, print the shape of the table, and then you're done